# 「球面上の巡回サンタ問題」を解いてみよう! サンタが街にやってくる

## 「世界各国を最小距離で巡るコース」を計算するWolfram言語スクリプトをPythonから呼ぶ

In [ ]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wlexpr, wl

session = WolframLanguageSession()

#session = WolframLanguageSession('/Applications/Wolfram Engine.app/Contents/MacOS/WolframKernel') 
#session = WolframLanguageSession('/Applications/Wolfram Engine.app/Contents/Resources/Wolfram Player.app/Contents/MacOS/WolframKernel')
#session = WolframLanguageSession('/Applications/Mathematica 12.3.1.app/Contents/MacOS/WolframKernel')

In [7]:
# 限られた紙面で収めるために、文字数削減のために定義した関数
def f(expression):
    return session.evaluate(wlexpr(expression))

In [19]:
# 「球面上で、世界各国を最小移動距離で一周する経路」を計算し、
# その経路を画像保存する
s=f('''places=CountryData["Countries"];
centers=Map[GeoPosition[CountryData[#,"CenterCoordinates"]]&,places];
{dist,route}=FindShortestTour[centers];
GeoGraphics[{Red,Thickness[0.005],GeoPath[centers[[route]]]}, 
GeoRange->"World",GeoBackground->GeoStyling["ReliefMap"],ImageSize->2400]''')
session.evaluate(wl.Export("worldmap.png",s,"PNG"))

'worldmap.png'

##  「世界各国を最小距離で巡る航路」の緯度・経度リスト作成

In [ ]:
# 「球面上で、世界各国を最小移動距離で一周する経路」の経路を緯度経度リストにする
f('''countryCenters=Map[First[GeoPositionXYZ[#]]&,centers[[route]]];
arc[x_,y_]:=Module[{a},a=VectorAngle[x,y];
Table[Evaluate[RotationTransform[\[Theta],{x,y}][x]],{\[Theta],0,a,a/Ceiling[10 a]}]]
tourLine=Apply[arc,Partition[countryCenters,2,1],{1}];
route=Map[QuantityMagnitude[LatitudeLongitude[GeoPositionXYZ[#]],"AngularDegrees"]&,  
Flatten[tourLine,1],{1}]''')
lat_lons =  f('route')

## 緯度・経度リストから、Google Earthで読み込むことができるKML形式ファイルを作成

In [9]:
# Google earthで可視化するために、前節記事の「航空機情報」形式に近づける
from collections import namedtuple
flights = []
FlightPos=namedtuple('FlightPos',('longitude','latitude','altitude'))
for lat_lon in lat_lons:
    flights.append(FlightPos(latitude=lat_lon[0],longitude=lat_lon[1],
                             altitude=100000))

In [12]:
# サンタの移動経路を、Google Earthで可視化する
fname = 'flight_path'
f = open(fname+'.kml', 'w')
#Writing the kml file.
f.write("<?xml version='1.0' encoding='UTF-8'?>\n")
f.write("<kml xmlns='http://earth.google.com/kml/2.2'>\n")
f.write("<Document>\n")
f.write("   <name>flight</name>\n")

for i in range(len(flights)-1):
    
    f.write("<Placemark>\n") 
    f.write("        <TimeSpan>\n          <begin>"
      +'2015-12-02T%02i:%02i:%02iZ'%(12,30,0)
      +"</begin>\n        </TimeSpan>\n")        
    f.write("   <Style>\n")
    f.write("   <LineStyle>\n")
    f.write("   <color>800000ff</color>\n") # abgr

    #f.write("   <width>0.2</width>\n")
    
    f.write("   <width>15</width>\n")
    
    f.write("   </LineStyle>\n")
    f.write("   </Style>\n")
    f.write("   <LineString>\n")
    f.write("       <extrude>1</extrude>\n")
    f.write("       <altitudeMode>absolute</altitudeMode>\n")
    f.write("        <coordinates>"
            
    #  +str(flight.longitude)+","+str(flight.latitude)+","+str(flight.altitude*0.3048)
    #  +" "
    #  +str(flight.longitude)+","+str(flight.latitude)+","+str(0)+"</coordinates>\n")
    
      +str(flights[i].longitude)+","+str(flights[i].latitude)+","+str(flights[i].altitude)
      +" "+str(flights[i+1].longitude)+","+str(flights[i+1].latitude)+","
      +str(flights[i].altitude)+"</coordinates>\n")
    
    f.write("   </LineString>\n")    
    f.write("</Placemark>\n")    
f.write("</Document>")
f.write("</kml>\n")
f.close()

## サンタ’[t]==こども’[t] という微分方程式を解いて、得られた解を画像で保存する

In [14]:
# サンタの微分方程式を解いて、得られた答えを画像として保存する
s = "eq=DSolve[サンタ'[t]==こども'[t],サンタ[t],t];" \
 + "Rasterize[eq // TraditionalForm,ImageSize->2400]"
session.evaluate(wl.Export("eq.png", f(s), "PNG"))

'eq.png'